# Import required libraries
---

In [1]:
import os
import gzip
import ast
import itertools
import logging
import torch
from typing import List, Any
import torch.nn.functional as F
import numpy as np
import pandas as pd
from allennlp.modules import ConditionalRandomField
from allennlp.modules.conditional_random_field import allowed_transitions
from torch import nn
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup, AutoModel
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from collections import defaultdict
from typing import Set
from overrides import overrides
from allennlp.training.metrics.metric import Metric
import argparse
import time
from pytorch_lightning import seed_everything
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, EarlyStopping
import warnings
from tqdm import tqdm
import random as rn

from pandarallel import pandarallel
# Initialization
pandarallel.initialize(progress_bar = True)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
conll_iob = {'B-ORG': 0, 'I-ORG': 1, 'B-MISC': 2, 'I-MISC': 3, 'B-LOC': 4, 
             'I-LOC': 5, 'B-PER': 6, 'I-PER': 7, 'O': 8}
wnut_iob = {'B-CORP': 0, 'I-CORP': 1, 'B-CW': 2, 'I-CW': 3, 'B-GRP': 4, 
            'I-GRP': 5, 'B-LOC': 6, 'I-LOC': 7, 'B-PER': 8, 'I-PER': 9, 
            'B-PROD': 10, 'I-PROD': 11, 'O': 12}
SEED = 42
rn.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

# Dataset Loading Functions and Class
---

In [3]:
def get_ner_reader(data):
    
    # 'fields' contains 4 lists 
    # The first list is the list of words present in the sentence
    # The last list is the list of ner tags of the words.
    
    fin = gzip.open(data, 'rt') if data.endswith('.gz') else open(data, 'rt')
    
    for is_divider, lines in itertools.groupby(fin, _is_divider):
        
        if is_divider:
            continue
        
        fields = [line.strip().split() for line in lines]
        fields = [list(field) for field in zip(*fields)]
        
        yield fields

# Function to assign the new tags 
def _assign_ner_tags(ner_tag, rep_):
    
    ner_tags_rep = []
    token_masks = []

    sub_token_len = len(rep_)
    token_masks.extend([True] * sub_token_len)
    
    if ner_tag[0] == 'B':
        
        in_tag = 'I' + ner_tag[1:]
        ner_tags_rep.append(ner_tag)
        ner_tags_rep.extend([in_tag] * (sub_token_len - 1))
    
    else:
        ner_tags_rep.extend([ner_tag] * sub_token_len)
    
    return ner_tags_rep, token_masks

# Function to extract spans (BI spans) and store in a dictionary
def extract_spans(tags):
    
    cur_tag = None
    cur_start = None
    gold_spans = {}

    def _save_span(_cur_tag, _cur_start, _cur_id, _gold_spans):
        
        if _cur_start is None:
            return _gold_spans
        
        _gold_spans[(_cur_start, _cur_id - 1)] = _cur_tag  # inclusive start & end, accord with conll-coref settings
        
        return _gold_spans

    # iterate over the tags
    for _id, nt in enumerate(tags):
        
        indicator = nt[0]
        
        if indicator == 'B':
            gold_spans = _save_span(cur_tag, cur_start, _id, gold_spans)
            cur_start = _id
            cur_tag = nt[2:]
            pass
        
        elif indicator == 'I':
            # do nothing
            pass
        
        elif indicator == 'O':
            gold_spans = _save_span(cur_tag, cur_start, _id, gold_spans)
            cur_tag = 'O'
            cur_start = _id
            pass
    
    _save_span(cur_tag, cur_start, _id + 1, gold_spans)
    
    return gold_spans


def _is_divider(line: str) -> bool:
    
    empty_line = line.strip() == ''
    
    if empty_line:
        return True

    first_token = line.split()[0]
    if first_token == "-DOCSTART-" or line.startswith('# id'):  # pylint: disable=simplifiable-if-statement
        return True

    return False

In [4]:
class CoNLLReader(Dataset):
    
    def __init__(self, max_instances = -1, max_length = 50, target_vocab = None, 
                 pretrained_dir = '', encoder_model = 'xlm-roberta-large', feature_type = 'encoded_noun_features'):
        
        self.feature_type = feature_type
        
        self._max_instances = max_instances
        self._max_length = max_length

        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_dir + encoder_model)

        self.pad_token = self.tokenizer.special_tokens_map['pad_token']
        self.pad_token_id = self.tokenizer.get_vocab()['pad']
        self.sep_token = self.tokenizer.special_tokens_map['sep_token']

        self.label_to_id = {} if target_vocab is None else target_vocab
        self.instances = []

    def get_target_size(self):
        return len(set(self.label_to_id.values()))

    def get_target_vocab(self):
        return self.label_to_id

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, item):
        return self.instances[item]
    
    def give_list(self, sentence):
        res = ast.literal_eval(sentence)
        return res
    
    def get_encoded_features(self, features_data):
        
        df = pd.read_csv(features_data)
        features_list = df[self.feature_type].parallel_apply(lambda x: self.give_list(x))
        return features_list
        
    def read_data(self, data, features_data):
        
        dataset_name = data if isinstance(data, str) else 'dataframe'
        print("Obtaining {} from {}".format(self.feature_type, features_data))
        features_list = self.get_encoded_features(features_data)
        
        print("Reading file {}".format(dataset_name))
        instance_idx = 0
        
        for fields, features in tqdm(zip(get_ner_reader(data = data), features_list)):
            
            if self._max_instances != -1 and instance_idx > self._max_instances:
                break
            
            sentence_str, tokens_sub_rep, token_masks_rep, coded_ner_, gold_spans_ = self.parse_line_for_ner(fields = fields)
            
            tokens_tensor = torch.tensor(tokens_sub_rep, dtype = torch.long)
            tag_tensor = torch.tensor(coded_ner_, dtype = torch.long).unsqueeze(0)
            token_masks_rep = torch.tensor(token_masks_rep)
            features_tensor = torch.tensor(features)
            
            self.instances.append((tokens_tensor, token_masks_rep, gold_spans_, tag_tensor, features_tensor))
            instance_idx += 1
                    
        print("Finished reading {:d} instances from file {}".format(len(self.instances), dataset_name))
    
    def parse_line_for_ner(self, fields):
        
        tokens_, ner_tags = fields[0], fields[-1]

        sentence_str, tokens_sub_rep, ner_tags_rep, token_masks_rep = self.parse_tokens_for_ner(tokens_, ner_tags)
        gold_spans_ = extract_spans(ner_tags_rep)
        coded_ner_ = [self.label_to_id[tag] for tag in ner_tags_rep]

        return sentence_str, tokens_sub_rep, token_masks_rep, coded_ner_, gold_spans_

    def parse_tokens_for_ner(self, tokens_, ner_tags):
        
        sentence_str = ''
        tokens_sub_rep, ner_tags_rep = [self.pad_token_id], ['O']
        
        for idx, token in enumerate(tokens_):
            
            if self._max_length != -1 and len(tokens_sub_rep) > self._max_length:
                break
            
            sentence_str += ' ' + ' '.join(self.tokenizer.tokenize(token.lower()))
            rep_ = self.tokenizer(token.lower())['input_ids']
            rep_ = rep_[1:-1]
            tokens_sub_rep.extend(rep_)

            # if we have a NER here, in the case of B, the first NER tag is the B tag, the rest are I tags.
            ner_tag = ner_tags[idx]
            tags, masks = _assign_ner_tags(ner_tag, rep_)
            ner_tags_rep.extend(tags)

        tokens_sub_rep.append(self.pad_token_id)
        ner_tags_rep.append('O')
        token_masks_rep = [True] * len(tokens_sub_rep)
        
        return sentence_str, tokens_sub_rep, ner_tags_rep, token_masks_rep

In [5]:
def get_tagset(tagging_scheme):
    if 'conll' in tagging_scheme:
        return conll_iob
    return wnut_iob

def get_reader(file_path, features_file_path, max_instances=-1, max_length=50, target_vocab=None, encoder_model='xlm-roberta-large', feature_type = 'encoded_noun_features'):
    if file_path is None:
        return None
    reader = CoNLLReader(max_instances=max_instances, max_length=max_length, target_vocab=target_vocab, encoder_model=encoder_model, feature_type = feature_type)
    reader.read_data(file_path, features_file_path)

    return reader

# Args Class
---

In [6]:
class Args():
    
    def __init__(self):
        
        self.train = '../input/semeval-task-11/EN-English/en_train.conll'
        self.test = '../input/semeval-task-11/EN-English/en_dev.conll'
        self.dev = '../input/semeval-task-11/EN-English/en_dev.conll'
        
        self.train_features = '../input/semeval-task-11/en_train_features.csv'
        self.test_features = '../input/semeval-task-11/en_dev_features.csv'
        self.dev_features = '../input/semeval-task-11/en_dev_features.csv'
        
        # Feature type will be either 'encoded_noun_features' or 'encoded_dependency_features'
        self.feature_type = 'encoded_dependency_features'
        self.feature_length = 44
        
        self.out_dir = './'
        self.iob_tagging = 'wnut'
        
        self.max_instances = -1
        self.max_length = 50
        
        # encoder_model can be bert-base-multilingual-cased or xlm-roberta-base or ai4bharat/indic-bert 
        self.encoder_model = 'bert-base-multilingual-cased'
        self.model = './'
        self.model_name = 'bert-base-multilingual-cased'
        self.stage = 'fit'
        self.prefix = 'test'

        self.batch_size = 32
        self.gpus = 1
        self.device = 'cuda'
        self.epochs = 3
        self.lr = 1e-5
        self.dropout = 0.1
        self.max_grad_norm = 1.0

In [7]:
sg = Args()

# Metric
---

In [8]:
class SpanF1(Metric):
    
    def __init__(self, non_entity_labels = ['O']) -> None:
        
        self._num_gold_mentions = 0
        self._num_recalled_mentions = 0
        self._num_predicted_mentions = 0
        self._TP, self._FP, self._GT = defaultdict(int), defaultdict(int), defaultdict(int)
        self.non_entity_labels = set(non_entity_labels)

    @overrides
    def __call__(self, batched_predicted_spans, batched_gold_spans, sentences = None):
        
        non_entity_labels = self.non_entity_labels

        for predicted_spans, gold_spans in zip(batched_predicted_spans, batched_gold_spans):
            gold_spans_set = set([x for x, y in gold_spans.items() if y not in non_entity_labels])
            pred_spans_set = set([x for x, y in predicted_spans.items() if y not in non_entity_labels])

            self._num_gold_mentions += len(gold_spans_set)
            self._num_recalled_mentions += len(gold_spans_set & pred_spans_set)
            self._num_predicted_mentions += len(pred_spans_set)

            for ky, val in gold_spans.items():
                if val not in non_entity_labels:
                    self._GT[val] += 1

            for ky, val in predicted_spans.items():
                if val in non_entity_labels:
                    continue
                if ky in gold_spans and val == gold_spans[ky]:
                    self._TP[val] += 1
                else:
                    self._FP[val] += 1

    @overrides
    def get_metric(self, reset: bool = False) -> float:
        
        all_tags: Set[str] = set()
        all_tags.update(self._TP.keys())
        all_tags.update(self._FP.keys())
        all_tags.update(self._GT.keys())
        all_metrics = {}

        for tag in all_tags:
            precision, recall, f1_measure = self.compute_prf_metrics(true_positives=self._TP[tag],
                                                                     false_negatives=self._GT[tag] - self._TP[tag],
                                                                     false_positives=self._FP[tag])
            all_metrics['P@{}'.format(tag)] = precision
            all_metrics['R@{}'.format(tag)] = recall
            all_metrics['F1@{}'.format(tag)] = f1_measure

        # Compute the precision, recall and f1 for all spans jointly.
        precision, recall, f1_measure = self.compute_prf_metrics(true_positives=sum(self._TP.values()),
                                                                 false_positives=sum(self._FP.values()),
                                                                 false_negatives=sum(self._GT.values())-sum(self._TP.values()))
        all_metrics["micro@P"] = precision
        all_metrics["micro@R"] = recall
        all_metrics["micro@F1"] = f1_measure

        if self._num_gold_mentions == 0:
            entity_recall = 0.0
        else:
            entity_recall = self._num_recalled_mentions / float(self._num_gold_mentions)

        if self._num_predicted_mentions == 0:
            entity_precision = 0.0
        else:
            entity_precision = self._num_recalled_mentions / float(self._num_predicted_mentions)

        all_metrics['MD@R'] = entity_recall
        all_metrics['MD@P'] = entity_precision
        all_metrics['MD@F1'] = 2. * ((entity_precision * entity_recall) / (entity_precision + entity_recall + 1e-13))
        all_metrics['ALLTRUE'] = self._num_gold_mentions
        all_metrics['ALLRECALLED'] = self._num_recalled_mentions
        all_metrics['ALLPRED'] = self._num_predicted_mentions
        if reset:
            self.reset()
        return all_metrics

    @staticmethod
    def compute_prf_metrics(true_positives: int, false_positives: int, false_negatives: int):
        
        precision = float(true_positives) / float(true_positives + false_positives + 1e-13)
        recall = float(true_positives) / float(true_positives + false_negatives + 1e-13)
        f1_measure = 2. * ((precision * recall) / (precision + recall + 1e-13))
        return precision, recall, f1_measure

    @overrides
    def reset(self):
        
        self._num_gold_mentions = 0
        self._num_recalled_mentions = 0
        self._num_predicted_mentions = 0
        self._TP.clear()
        self._FP.clear()
        self._GT.clear()

# Model class
---

In [9]:
class NERModel(nn.Module):
    
    def __init__(self,
                 lr = 1e-5,
                 dropout_rate = 0.1,
                 batch_size = 16,
                 tag_to_id = None,
                 stage = 'fit',
                 pad_token_id = 1,
                 feature_length = 44,
                 encoder_model = 'xlm-roberta-large',
                 num_gpus = 1):
        super(NERModel, self).__init__()

        self.id_to_tag = {v: k for k, v in tag_to_id.items()}
        self.tag_to_id = tag_to_id
        self.batch_size = batch_size

        self.stage = stage
        self.num_gpus = num_gpus
        self.target_size = len(self.id_to_tag)
        self.feature_length = feature_length
        
        # set the default baseline model here
        self.pad_token_id = pad_token_id

        self.encoder_model = encoder_model
        self.encoder = AutoModel.from_pretrained(encoder_model, return_dict = True)
        
        self.feedforward = nn.Linear(in_features = self.encoder.config.hidden_size + self.feature_length, 
                                     out_features = self.target_size)
        
        self.crf_layer = ConditionalRandomField(num_tags = self.target_size, 
                                                constraints = allowed_transitions(constraint_type = "BIO", 
                                                                                  labels = self.id_to_tag))

        self.lr = lr
        self.dropout = nn.Dropout(dropout_rate)

        self.span_f1 = SpanF1()        

    def forward(self, batch):
        
        tokens, tags, token_mask, metadata, features = batch
        batch_size = tokens.size(0)
        
        embedded_text_input = self.encoder(input_ids=tokens, attention_mask=token_mask)
        embedded_text_input = embedded_text_input.last_hidden_state
        features = features.unsqueeze_(-1)
        features = features.expand(features.shape[0], features.shape[1], embedded_text_input.shape[1])
        features = features.reshape((features.shape[0], features.shape[2], features.shape[1]))
        embedded_text_input = torch.cat((embedded_text_input, features), dim = 2)
        embedded_text_input = self.dropout(F.leaky_relu(embedded_text_input))

        # project the token representation for classification
        token_scores = self.feedforward(embedded_text_input)
   
        # compute the log-likelihood loss and compute the best NER annotation sequence
        output = self._compute_token_tags(token_scores=token_scores, tags=tags, token_mask=token_mask, metadata=metadata, batch_size=batch_size)
        return output
    
    def _compute_token_tags(self, token_scores, tags, token_mask, metadata, batch_size):
        
        # compute the log-likelihood loss and compute the best NER annotation sequence
        loss = -self.crf_layer(token_scores, tags, token_mask) / float(batch_size)
        best_path = self.crf_layer.viterbi_tags(token_scores, token_mask)

        pred_results = []
        for i in range(batch_size):
            tag_seq, _ = best_path[i]
            pred_results.append(extract_spans([self.id_to_tag[x] for x in tag_seq if x in self.id_to_tag]))

        self.span_f1(pred_results, metadata)
        output = {"loss": loss, "results": self.span_f1.get_metric()}
        return output

# Function to load train and validation data
---

In [10]:
def dataloading():
    train_data = get_reader(file_path=sg.train, features_file_path = sg.train_features,
                            target_vocab=get_tagset(sg.iob_tagging), 
                            encoder_model=sg.encoder_model, max_instances=sg.max_instances,
                            max_length=sg.max_length, feature_type = sg.feature_type)
    dev_data = get_reader(file_path=sg.dev, features_file_path = sg.dev_features, 
                          target_vocab=get_tagset(sg.iob_tagging), 
                          encoder_model=sg.encoder_model, max_instances=sg.max_instances, 
                          max_length=sg.max_length, feature_type = sg.feature_type)

    return train_data, dev_data

In [11]:
train_data, dev_data = dataloading()

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Obtaining encoded_dependency_features from ../input/semeval-task-11/en_train_features.csv


...

15300it [00:47, 320.53it/s]


Finished reading 15300 instances from file ../input/semeval-task-11/EN-English/en_train.conll
Obtaining encoded_dependency_features from ../input/semeval-task-11/en_dev_features.csv


Reading file ../input/semeval-task-11/EN-English/en_dev.conll


800it [00:02, 299.14it/s]

Finished reading 800 instances from file ../input/semeval-task-11/EN-English/en_dev.conll


# Prepare data into batches for training and evaluation
---

In [12]:
def collate_batch(batch):
        
        batch_ = list(zip(*batch))
        tokens, masks, gold_spans, tags, features = batch_[0], batch_[1], batch_[2], batch_[3], batch_[4]

        max_len = max([len(token) for token in tokens])
        token_tensor = torch.empty(size = (len(tokens), max_len), 
                                   dtype = torch.long).fill_(1)
        tag_tensor = torch.empty(size = (len(tokens), max_len), 
                                 dtype = torch.long).fill_(model.tag_to_id['O'])
        mask_tensor = torch.zeros(size = (len(tokens), max_len), dtype = torch.bool)
        
        features = torch.stack(list(features), dim=0)
            
        for i in range(len(tokens)):
            
            tokens_ = tokens[i]
            seq_len = len(tokens_)

            token_tensor[i, :seq_len] = tokens_
            tag_tensor[i, :seq_len] = tags[i]
            mask_tensor[i, :seq_len] = masks[i]

        return token_tensor, tag_tensor, mask_tensor, gold_spans, features

In [13]:
def train_dataloader():
    loader = DataLoader(train_data, batch_size = sg.batch_size, collate_fn = collate_batch, num_workers = 1)
    return loader

def val_dataloader():
    loader = DataLoader(dev_data, batch_size = sg.batch_size, collate_fn = collate_batch, num_workers = 1)
    return loader

In [14]:
training_dataloader = train_dataloader()
validation_dataloader = val_dataloader()

In [15]:
model = NERModel(tag_to_id = train_data.get_target_vocab(), dropout_rate = sg.dropout, 
                 batch_size = sg.batch_size, stage = sg.stage, lr = sg.lr, feature_length = sg.feature_length,
                         encoder_model = sg.encoder_model, num_gpus = sg.gpus)

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [16]:
model.to(sg.device)

NERModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=sg.lr)

# Training and Evaluation
---

In [18]:
def train_and_evaluate():
    
    print("----------------------- Training ----------------------------")
    print()
    
    # Training loop
    for epoch_i in tqdm(range(sg.epochs)):

        epoch_iterator = tqdm(training_dataloader, desc = "Iteration", position = 0, leave = True)

        # TRAIN loop
        model.train()
        training_loss = 0

        for step, batch in enumerate(epoch_iterator):
            #print(batch)
            #print(batch[4][32])
            batch = (batch[0].to(sg.device), batch[1].to(sg.device), batch[2].to(sg.device), batch[3], batch[4].to(sg.device))
            # forward pass
            output = model.forward(batch)

            # backward pass
            loss = output['loss']
            loss.backward()

            # track train loss
            training_loss += loss.item()

            # gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = sg.max_grad_norm)

            # update parameters
            optimizer.step()

        # print train loss per epoch
        training_loss = training_loss / len(training_dataloader)
        print()
        print('Epoch: {} \tTraining Loss: {:.5f}'.format(epoch_i + 1, training_loss))
    
        metric_scores = model.span_f1.get_metric()
        model.span_f1.reset()
        
        print()
        print("Epoch: {} metrics".format(epoch_i+1))
        print()
        for key, value in metric_scores.items():
            print("{}: {:.5f},".format(key, value), end = " ")
        print()
    
    print()
    print("--------------------- Evaluation ---------------------")
    print()
    
    # Loop for evaluation on validation set
    
    epoch_iterator = tqdm(validation_dataloader, desc = "Iteration", position = 0, leave = True)
    
    validation_loss = 0
    for step, batch in enumerate(epoch_iterator):
        
        batch = (batch[0].to(sg.device), batch[1].to(sg.device), batch[2].to(sg.device), batch[3], batch[4].to(sg.device))

        with torch.no_grad():
            output = model.forward(batch)

        loss = output['loss']
        validation_loss += loss.item()

    validation_loss = validation_loss / len(validation_dataloader)
    print()
    print('Validation Loss: {:.5f}'.format(validation_loss))
    print()
    metric_scores = model.span_f1.get_metric()
    model.span_f1.reset()
    print()
    print("Metrics on validation set")
    print()
    for key, value in metric_scores.items():
        print("{}: {:.5f},".format(key, value), end = " ")
    print()
    print()
    torch.save(model, "./" + sg.model_name + "_" + str(sg.batch_size) + "_" + str(sg.lr) + ".pt")
    print("Saved the model")

In [19]:
train_and_evaluate()

----------------------- Training ----------------------------



 33%|███▎      | 1/3 [02:29<04:59, 149.50s/it]


Epoch: 1 	Training Loss: 19.45221

Epoch: 1 metrics

P@CORP: 0.20664, R@CORP: 0.32401, F1@CORP: 0.25235, P@PER: 0.55120, R@PER: 0.66630, F1@PER: 0.60331, P@LOC: 0.46962, R@LOC: 0.51219, F1@LOC: 0.48998, P@PROD: 0.22087, R@PROD: 0.12385, F1@PROD: 0.15870, P@GRP: 0.31662, R@GRP: 0.39793, F1@GRP: 0.35265, P@CW: 0.44152, R@CW: 0.25053, F1@CW: 0.31967, micro@P: 0.39310, micro@R: 0.41545, micro@F1: 0.40396, MD@R: 0.51463, MD@P: 0.48694, MD@F1: 0.50040, ALLTRUE: 23553.00000, ALLRECALLED: 12121.00000, ALLPRED: 24892.00000, 


 67%|██████▋   | 2/3 [04:59<02:29, 149.71s/it]


Epoch: 2 	Training Loss: 5.31680

Epoch: 2 metrics

P@CORP: 0.66476, R@CORP: 0.67117, F1@CORP: 0.66795, P@PER: 0.89349, R@PER: 0.91866, F1@PER: 0.90590, P@LOC: 0.81334, R@LOC: 0.81809, F1@LOC: 0.81571, P@PROD: 0.47260, R@PROD: 0.47212, F1@PROD: 0.47236, P@GRP: 0.73007, R@GRP: 0.69756, F1@GRP: 0.71345, P@CW: 0.59101, R@CW: 0.57463, F1@CW: 0.58270, micro@P: 0.72345, micro@R: 0.72173, micro@F1: 0.72259, MD@R: 0.79629, MD@P: 0.79819, MD@F1: 0.79724, ALLTRUE: 23553.00000, ALLRECALLED: 18755.00000, ALLPRED: 23497.00000, 


100%|██████████| 3/3 [07:28<00:00, 149.43s/it]



Epoch: 3 	Training Loss: 3.61308

Epoch: 3 metrics

P@CORP: 0.75785, R@CORP: 0.76760, F1@CORP: 0.76270, P@PER: 0.92749, R@PER: 0.94089, F1@PER: 0.93414, P@LOC: 0.86488, R@LOC: 0.87497, F1@LOC: 0.86990, P@PROD: 0.59868, R@PROD: 0.61854, F1@PROD: 0.60845, P@GRP: 0.79753, R@GRP: 0.76001, F1@GRP: 0.77832, P@CW: 0.70059, R@CW: 0.69350, F1@CW: 0.69703, micro@P: 0.79554, micro@R: 0.79773, micro@F1: 0.79663, MD@R: 0.85386, MD@P: 0.85151, MD@F1: 0.85268, ALLTRUE: 23553.00000, ALLRECALLED: 20111.00000, ALLPRED: 23618.00000, 

--------------------- Evaluation ---------------------



Iteration: 100%|██████████| 25/25 [00:03<00:00,  6.35it/s]



Validation Loss: 5.51213


Metrics on validation set

P@CORP: 0.78528, R@CORP: 0.66321, F1@CORP: 0.71910, P@PER: 0.88562, R@PER: 0.93448, F1@PER: 0.90940, P@LOC: 0.73801, R@LOC: 0.85470, F1@LOC: 0.79208, P@PROD: 0.54348, R@PROD: 0.51020, F1@PROD: 0.52632, P@GRP: 0.76506, R@GRP: 0.66842, F1@GRP: 0.71348, P@CW: 0.57831, R@CW: 0.54545, F1@CW: 0.56140, micro@P: 0.74132, micro@R: 0.72927, micro@F1: 0.73525, MD@R: 0.80894, MD@P: 0.82231, MD@F1: 0.81557, ALLTRUE: 1230.00000, ALLRECALLED: 995.00000, ALLPRED: 1210.00000, 

Saved the model
